In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 22: Grid Computing ---</h2><p>You gain access to a massive storage cluster arranged in a grid; each storage node is only connected to the four nodes directly adjacent to it (three if the node is on an edge, two if it's in a corner).</p>
<p>You can directly access data <em>only</em> on node <code>/dev/grid/node-x0-y0</code>, but you can perform some limited actions on the other nodes:</p>
<ul>
<li>You can get the disk usage of all nodes (via <a href="https://en.wikipedia.org/wiki/Df_(Unix)#Example"><code>df</code></a>). The result of doing this is in your puzzle input.</li>
<li>You can instruct a node to <span title="You suspect someone misunderstood the x86 MOV instruction."><em>move</em></span> (not copy) <em>all</em> of its data to an adjacent node (if the destination node has enough space to receive the data). The sending node is left empty after this operation.</li>
</ul>
<p>Nodes are named by their position: the node named <code>node-x10-y10</code> is adjacent to nodes <code>node-x9-y10</code>, <code>node-x11-y10</code>, <code>node-x10-y9</code>, and <code>node-x10-y11</code>.</p>
<p>Before you begin, you need to understand the arrangement of data on these nodes. Even though you can only move data between directly connected nodes, you're going to need to rearrange a lot of the data to get access to the data you need.  Therefore, you need to work out how you might be able to shift data around.</p>
<p>To do this, you'd like to count the number of <em>viable pairs</em> of nodes.  A viable pair is any two nodes (A,B), <em>regardless of whether they are directly connected</em>, such that:</p>
<ul>
<li>Node A is <em>not</em> empty (its <code>Used</code> is not zero).</li>
<li>Nodes A and B are <em>not the same</em> node.</li>
<li>The data on node A (its <code>Used</code>) <em>would fit</em> on node B (its <code>Avail</code>).</li>
</ul>
<p><em>How many viable pairs</em> of nodes are there?</p>
</article>


In [2]:
from itertools import islice


def viable_pairs(df: str) -> int:
    nodes = []
    for line in islice(df.strip().splitlines(), 2, None):
        _, _, used, avail, _ = line.split()
        used, avail = int(used[:-1]), int(avail[:-1])
        nodes.append((used, avail))

    count = 0
    for node_a, node_b in product(nodes, repeat=2):
        if node_a is not node_b:
            (u_a, _), (_, a_b) = node_a, node_b
            if u_a and u_a <= a_b:
                count += 1

    return count


with open("../input/day22.txt") as f:
    df = f.read()

viable_pairs(df)

955

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>955</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now that you have a better understanding of the grid, it's time to get to work.</p>
<p>Your goal is to gain access to the data which begins in the node with <code>y=0</code> and the <em>highest <code>x</code></em> (that is, the node in the top-right corner).</p>
<p>For example, suppose you have the following grid:</p>
<pre><code>Filesystem            Size  Used  Avail  Use%
/dev/grid/node-x0-y0   10T    8T     2T   80%
/dev/grid/node-x0-y1   11T    6T     5T   54%
/dev/grid/node-x0-y2   32T   28T     4T   87%
/dev/grid/node-x1-y0    9T    7T     2T   77%
/dev/grid/node-x1-y1    8T    0T     8T    0%
/dev/grid/node-x1-y2   11T    7T     4T   63%
/dev/grid/node-x2-y0   10T    6T     4T   60%
/dev/grid/node-x2-y1    9T    8T     1T   88%
/dev/grid/node-x2-y2    9T    6T     3T   66%
</code></pre>
<p>In this example, you have a storage grid <code>3</code> nodes wide and <code>3</code> nodes tall.  The node you can access directly, <code>node-x0-y0</code>, is almost full. The node containing the data you want to access, <code>node-x2-y0</code> (because it has <code>y=0</code> and the highest <code>x</code> value), contains 6 <a href="https://en.wikipedia.org/wiki/Terabyte">terabytes</a> of data - enough to fit on your node, if only you could make enough space to move it there.</p>
<p>Fortunately, <code>node-x1-y1</code> looks like it has enough free space to enable you to move some of this data around.  In fact, it seems like all of the nodes have enough space to hold any node's data (except <code>node-x0-y2</code>, which is much larger, very full, and not moving any time soon). So, initially, the grid's capacities and connections look like this:</p>
<pre><code>( 8T/10T) --  7T/ 9T -- [ 6T/10T]
    |           |           |
  6T/11T  --  0T/ 8T --   8T/ 9T
    |           |           |
 28T/32T  --  7T/11T --   6T/ 9T
</code></pre>
<p>The node you can access directly is in parentheses; the data you want starts in the node marked by square brackets.</p>
<p>In this example, most of the nodes are interchangable: they're full enough that no other node's data would fit, but small enough that their data could be moved around. Let's draw these nodes as <code>.</code>. The exceptions are the empty node, which we'll draw as <code>_</code>, and the very large, very full node, which we'll draw as <code>#</code>. Let's also draw the goal data as <code>G</code>. Then, it looks like this:</p>
<pre><code>(.) .  G
 .  _  .
 #  .  .
</code></pre>
<p>The goal is to move the data in the top right, <code>G</code>, to the node in parentheses. To do this, we can issue some commands to the grid and rearrange the data:</p>
<ul>
<li><p>Move data from <code>node-y0-x1</code> to <code>node-y1-x1</code>, leaving node <code>node-y0-x1</code> empty:</p><pre><code>(.) _  G
 .  .  .
 #  .  .
</code></pre>
</li><li><p>Move the goal data from <code>node-y0-x2</code> to <code>node-y0-x1</code>:</p><pre><code>(.) G  _
 .  .  .
 #  .  .
</code></pre>
</li><li><p>At this point, we're quite close. However, we have no deletion command, so we have to move some more data around. So, next, we move the data from <code>node-y1-x2</code> to <code>node-y0-x2</code>:</p><pre><code>(.) G  .
 .  .  _
 #  .  .
</code></pre>
</li><li><p>Move the data from <code>node-y1-x1</code> to <code>node-y1-x2</code>:</p><pre><code>(.) G  .
 .  _  .
 #  .  .
</code></pre>
</li><li><p>Move the data from <code>node-y1-x0</code> to <code>node-y1-x1</code>:</p><pre><code>(.) G  .
 _  .  .
 #  .  .
</code></pre>
</li><li><p>Next, we can free up space on our node by moving the data from <code>node-y0-x0</code> to <code>node-y1-x0</code>:</p><pre><code>(_) G  .
 .  .  .
 #  .  .
</code></pre>
</li>
<li><p>Finally, we can access the goal data by moving the it from <code>node-y0-x1</code> to <code>node-y0-x0</code>:</p><pre><code>(G) _  .
 .  .  .
 #  .  .
</code></pre>
</li>
</ul>
<p>So, after <code>7</code> steps, we've accessed the data we want. Unfortunately, each of these moves takes time, and we need to be efficient:</p>
<p><em>What is the fewest number of steps</em> required to move your goal data to <code>node-x0-y0</code>?</p>
</article>

</main>


In [12]:
from collections import deque
from copy import deepcopy
from re import findall

from more_itertools import take
from numpy import empty
from tabulate import tabulate


example_grid = """
root@ebhq-gridcenter# df -h
Filesystem            Size  Used  Avail  Use%
/dev/grid/node-x0-y0   10T    8T     2T   80%
/dev/grid/node-x0-y1   11T    6T     5T   54%
/dev/grid/node-x0-y2   32T   28T     4T   87%
/dev/grid/node-x1-y0    9T    7T     2T   77%
/dev/grid/node-x1-y1    8T    0T     8T    0%
/dev/grid/node-x1-y2   11T    7T     4T   63%
/dev/grid/node-x2-y0   10T    6T     4T   60%
/dev/grid/node-x2-y1    9T    8T     1T   88%
/dev/grid/node-x2-y2    9T    6T     3T   66%
"""


def create_grid(grid: str) -> list[list[tuple[int, int]]]:
    nodes = []
    for line in islice(grid.strip().splitlines(), 2, None):
        node, size, used, *_ = line.split()
        used, size = int(used[:-1]), int(size[:-1])

        _, y = (int(i) for i in findall(r"\d+", node))

        if y == len(nodes):
            nodes.append([])
        nodes[y].append([used, size])
    return nodes


print(tabulate(create_grid(example_grid), tablefmt="rounded_grid"))


def min_steps(grid: str) -> int:
    directions = ((-1, 0), (0, 1), (1, 0), (0, -1))
    grid = create_grid(grid)
    rows, cols = len(grid), len(grid[0])
    print(f"{rows=}, {cols=}")

    empty = take(
        1, ((r, c) for r, c in product(range(rows), range(cols)) if grid[r][c][0] == 0)
    )[0]
    print(f"{empty=}")

    queue = deque([((0, cols - 1), empty, grid)])
    seen = set(str(grid))

    steps = 0
    while queue:
        print(f"{steps=} and {len(queue)=}")
        for _ in range(len(queue)):
            (data_r, data_c), (empty_r, empty_c), state = queue.popleft()
            if data_r == 0 and data_c == 0:
                return steps

            delta_r, delta_c = -1, 0

            for delta_r, delta_c in directions:
                if 0 <= empty_r + delta_r < rows and 0 <= empty_c + delta_c < cols:
                    size = state[empty_r][empty_c][1]
                    used = state[empty_r + delta_r][empty_c + delta_c][0]
                    if used <= size:
                        state1 = deepcopy(state)
                        state1[empty_r + delta_r][empty_c + delta_c][0] = 0
                        state1[empty_r][empty_c][0] += used
                        if data_r == empty_r + delta_r and data_c == empty_c + delta_c:
                            data_r1, data_c1 = empty_r, empty_c
                        else:
                            data_r1, data_c1 = data_r, data_c

                        h = str(state1)
                        if h not in seen:
                            seen.add(h)
                            queue.append(
                                (
                                    (data_r1, data_c1),
                                    (empty_r + delta_r, empty_c + delta_c),
                                    state1,
                                )
                            )

        steps += 1

    return inf


def simplify(grid: str) -> list[list[str]]:
    grid = create_grid(grid)
    rows, cols = len(grid), len(grid[0])
    e_r, e_c = take(
        1, ((r, c) for r, c in product(range(rows), range(cols)) if grid[r][c][0] == 0)
    )[0]
    print(f"{(e_r, e_c)=}")
    simple = [["."] * cols for _ in range(rows)]
    to_be_moved = grid[0][-1][0]
    for r, c in product(range(rows), range(cols)):
        used, _ = grid[r][c]
        if used > 3 * to_be_moved:
            simple[r][c] = "#"
    simple[0][0] = "(.)"
    simple[0][-1] = "G"
    simple[e_r][e_c] = "_"
    return simple


print(tabulate(simplify(example_grid)))
print(f"{min_steps(example_grid)=}")

╭──────────┬─────────┬─────────╮
│ [8, 10]  │ [7, 9]  │ [6, 10] │
├──────────┼─────────┼─────────┤
│ [6, 11]  │ [0, 8]  │ [8, 9]  │
├──────────┼─────────┼─────────┤
│ [28, 32] │ [7, 11] │ [6, 9]  │
╰──────────┴─────────┴─────────╯
(e_r, e_c)=(1, 1)
---  -  -
(.)  .  G
.    _  .
#    .  .
---  -  -
rows=3, cols=3
empty=(1, 1)
steps=0 and len(queue)=1
steps=1 and len(queue)=4
steps=2 and len(queue)=7
steps=3 and len(queue)=6
steps=4 and len(queue)=10
steps=5 and len(queue)=22
steps=6 and len(queue)=29
steps=7 and len(queue)=34
min_steps(example_grid)=7


In [20]:
# print(tabulate(create_grid(df), tablefmt="rounded_grid"))
print(tabulate(simplify(df)))
# min_steps(df)

(e_r, e_c)=(23, 16)
---  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
(.)  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  G
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .

In [5]:
"""
--  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
(.)  .  .  .  . 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66
.    .  .  .  . 33  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  . 32  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  . 31  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  . 30  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  . 29  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  . 28  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  . 27  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  . 26  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  . 25  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  . 24  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  . 23  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  . 22  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #  #
.    .  .  .  . 21 20 19 18 17 16 15 14 13 12 11 10  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  9  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  8  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  7  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  6  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  5  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  4  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  3  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  2  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  1  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  0  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
.    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
---  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
"""

'\n--  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -\n(.)  .  .  .  . 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66\n.    .  .  .  . 33  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .\n.    .  .  .  . 32  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .\n.    .  .  .  . 31  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .\n.    .  .  .  . 30  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .\n.    .  .  .  . 29  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .\n.    .  .  .  . 28  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .\n.    .  .  .  . 27  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 

In [24]:
rows = 26
cols = 38
print("steps needed:", 66 + cols * 5, "IS NOT THE RIGHT ANSWER")
print("steps needed:", 66 + (cols - 1) * 5, "IS NOT THE RIGHT ANSWER")
print(
    "steps needed:",
    66 + (cols - 2) * 5,
    "The Right Answer Did this wrong because I start on start - 1 = G.col-1",
)

steps needed: 256 IS NOT THE RIGHT ANSWER
steps needed: 251 IS NOT THE RIGHT ANSWER
steps needed: 246 The Right Answer Did this wrong because I start on start - 1 = G.col-1


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>246</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
